#### Date: Jun 2019 (*Review: March 2024*)

#### The notebook proceeds in the following steps:
<br>1. It loads an example audio file containing a drum recording
<br>2. It computes the STFT of the audio data.
<br>3. It applies NMFD as described in [1], with audio-informed initialization of the components
<br>4. It visualizes the decomposition results.
<br>5. It resynthesizes the separated audio streams and saves them as wav files to the hard drive.

In [ ]:
import os
import numpy as np
import scipy.io.wavfile as wav
import IPython.display as ipd

from libnmfd.core.nmfconv import init_activations, init_templates, nmfd
from libnmfd.dsp.filters import alpha_wiener_filter
from libnmfd.dsp.transforms import forward_stft, inverse_stft
from libnmfd.utils import make_monaural, pcm_int16_to_float32np
from libnmfd.utils.core_utils import visualize_components_nmf

INPUT_DIR = 'data/'
OUT_DIR = 'output/'

# create the output directory if it doesn't exist
if not os.path.isdir(OUT_DIR):
    os.makedirs(OUT_DIR)

# convert wav from int16 to float32
filename = 'runningExample_AmenBreak.wav'

### 1. Load the audio signal

In [ ]:
fs, x = wav.read(os.path.join(INPUT_DIR, filename))

# make monaural if necessary
x = make_monaural(x)

x = pcm_int16_to_float32np(x)

### 2. compute STFT

In [ ]:
# spectral parameters
BLOCK_SIZE = 2048
HOP_SIZE = 512

# STFT computation
X, A, P = forward_stft(x, block_size=BLOCK_SIZE, hop_size=HOP_SIZE, reconst_mirror=True, append_frames=True)

# get dimensions and time and freq resolutions
num_bins, num_frames = X.shape
time_res = HOP_SIZE / fs
freq_res = fs / BLOCK_SIZE

### 3. Apply NMF variants to STFT magnitude

In [ ]:
# set common parameters
num_comp = 3
num_iter = 30
num_template_frames = 8

# generate initial guess for templates
init_W = init_templates(num_comp=num_comp,
                        num_bins=num_bins,
                        strategy='drums')



# generate initial activations
init_H = init_activations(num_comp=num_comp,
                          num_frames=num_frames,
                          strategy='uniform')

# NMFD core method
nmfd_W, nmfd_H, nmfd_V, divKL, _ = nmfd(V=A, 
                                        num_comp=num_comp, 
                                        num_frames=num_frames, 
                                        num_iter=num_iter,
                                        num_template_frames=num_template_frames,
                                        init_W=init_W,
                                        init_H=init_H)


# alpha-Wiener filtering
nmfd_A, _ = alpha_wiener_filter(A, nmfd_V, 1.0)

In [ ]:
#visualize
fh1, _ = visualize_components_nmf(V=A, W=nmfd_W, H=nmfd_H, comp_V=nmfd_A, time_res=time_res,
                                  freq_res=freq_res, end_sec=3.8, font_size=14)

In [ ]:
audios = []

# resynthesize results of NMF with soft constraints and score information
for k in range(num_comp):
    Y = nmfd_A[k] * np.exp(1j * P);
    y, _ = inverse_stft(X=Y,
                        block_size=BLOCK_SIZE,
                        hop_size=HOP_SIZE,
                        reconst_mirror=True,
                        append_frames=True,
                        num_samp=len(x))
    audios.append(y)
    
    # save result
    out_filepath = os.path.join(OUT_DIR,
                                'Winstons_AmenBreak_NMFD_component_{}.wav'.format(k, filename))
    
    wav.write(filename=out_filepath, rate=fs, data=y)

#### Breakbeat with 3 components

In [ ]:
ipd.Audio(x, rate=fs)

#### Kick Drum

In [ ]:
ipd.Audio(audios[0].T, rate=fs)

#### Snare Drum

In [ ]:
ipd.Audio(audios[1].T, rate=fs)

#### Ride Cymbal

In [ ]:
ipd.Audio(audios[2].T, rate=fs)

#### Reference: 
[1] Christian Dittmar, Meinard Müller
<br>**Reverse Engineering the Amen Break — Score-Informed Separation and Restoration Applied to Drum Recordings**
<br>IEEE/ACM Transactions on Audio, Speech, and Language Processing, 24(9): 1531-1543, 2016.

#### If you use the 'NMF toolbox' please refer to:
[2] Patricio López-Serrano, Christian Dittmar, Yiğitcan Özer, and Meinard Müller<br>
**NMF Toolbox: Music Processing Applications of Nonnegative Matrix Factorization**<br>
In Proceedings of the  International Conference on Digital Audio Effects (DAFx), 2019.